In [10]:
import json
import pennylane as qml
import pennylane.numpy as np
# Write any helper functions you need here

def GHZ_circuit(noise_param, n_qubits):

    """
    Quantum circuit that prepares an imperfect GHZ state using gates native to a neutral atom device.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a 
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the prepared GHZ state.
    Returns:
        - (np.tensor): A density matrix, as returned by `qml.state`, representing the imperfect GHZ state.
    
    """
    


    def circuit():

        for i in range(0, n_qubits):
            qml.RY(np.pi/2, wires=i)

        for i in range(1, n_qubits):
            qml.CZ(wires=[0, i])  # Entangle the qubits
            qml.DepolarizingChannel(noise_param, wires=i)  # Apply noise

        qml.RX(np.pi/2, wires=0)

        for i in range(1, n_qubits):
            qml.RY(np.pi/2,wires=i)

        qml.RX(np.pi/2, wires=0)

        return qml.state()
    
    return circuit()    

def GHZ_fidelity(noise_param, n_qubits):

    """
    Calculates the fidelity between the imperfect GHZ state returned by GHZ_circuit and the ideal GHZ state.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a 
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the GHZ state.
    Returns:
        - (float): The fidelity between the noisy and ideal GHZ states.
    """
    
    dev = qml.device('default.mixed', wires=n_qubits)
    GHZ_QNode = qml.QNode(GHZ_circuit, dev)
    noisy_state = GHZ_QNode(noise_param, n_qubits)  # Execute the GHZ_circuit to get the noisy state

    # Ideal GHZ state preparation
    ideal_state_vector = np.zeros((2**n_qubits,))
    ideal_state_vector[0] = ideal_state_vector[-1] = 1 / np.sqrt(2)
    ideal_state = np.outer(ideal_state_vector, ideal_state_vector.conj())
    
    return qml.math.fidelity(noisy_state, ideal_state)

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = GHZ_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    dev = qml.device('default.mixed', wires=4)
    qnode = qml.QNode(GHZ_circuit, dev)
    u = qnode(0.05,3)
    
    for op in qnode.tape.operations:
        assert (isinstance(op, qml.RX) or isinstance(op, qml.RY) or isinstance(op, qml.CZ) or isinstance(op, qml.DepolarizingChannel)), "You are using forbidden gates!"

    assert np.isclose(solution_output, expected_output, rtol = 1e-4)